An analysis on creating the joint positions / time for our quad wave gait.

# Highest Imports

Set up matplotlib

In [13]:
%matplotlib widget
import matplotlib.pyplot as plt

Other useful imports

In [14]:
import numpy as np

# Gait Parameters

In [15]:
beta = duty_cycle = 0.75
v = body_velocity = 0.04  # m/s
T = cycle_time = 1  # s
L = v * T # m

Compute $T_l$, the relative cycle time for each leg

In [16]:
T_l = (1 - beta) * T

# Generate Horizontal + Vertical Positions & Velocites

We will split this up into 6 checkpoints, similar to how it was done in the video. 
1. vertical launch (when the leg is lifting)
2. horizontal launch (leg is done lifting up and starts moving horizontally)
3. horizontal movement (leg is moving the final position)
4. horizontal movement stop
5. horizontal stop
6. vertical stop

Observe that the horizontal velocity graph is always symmetric, so we can split our checkpoints into the two following groups:
1. horizontal_move_start
2. horizontal_move_stable

As the leg always needs to move first, we know that $launch_{vertical} = 0$ always. Keep them as fractions of the **half cycle** as everything is symmetric and dynamic

In [17]:
horizontal_move_start = .4
horizontal_move_stable = .8
checkpoint_fracs = np.array((0, horizontal_move_start, horizontal_move_stable))
left = checkpoint_fracs * T_l / 2
right = T_l - left 
t = np.concatenate((left, np.flip(right)))
t

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25])

Visualize the checkpoints

In [18]:
plt.figure()
plt.scatter(t, np.full(t.shape, 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Horizontal Movement Over Time